In [294]:
import pandas as pd

In [295]:
# def read_dataset(filepath):
#     Transactions = pd.read_csv(filepath, header=None)
#     Transactions = Transactions.map(str)
#     Transactions = Transactions.apply(sorted, axis=1)
#     # print(Transactions.head())

#     return Transactions

def read_dataset(filepath):
    with open(filepath, 'r') as file:
        # Read the file and split each transaction by spaces
        transactions = [line.strip().split() for line in file.readlines()]
    return transactions

In [296]:
from collections import defaultdict

def find_frequent_1_itemsets(transactions, min_sup):
    item_count = defaultdict(int)
    for transaction in transactions:
        for item in set(transaction):  # Ensure that we count unique items
            item_count[item] += 1
    
    L1 = set()
    for item, count in item_count.items():
        if count >= min_sup:
            L1.add(tuple([item]))  # Create a frozenset of the frequent items
    

    return sorted(list(L1), key=lambda x: sorted(x))


In [297]:

def has_infrequent_subset(candidate, prev_frequent_itemsets):

    for item in candidate:
        subset = candidate.copy()
        subset.remove(item)
        if subset not in prev_frequent_itemsets:
            return True  # If any subset is not frequent, return True
    return False  # Otherwise, return False


def apriori_gen(Lk_minus_1 : list[list[str]], k_1:int):
    candidates = []
    Lk_minus_1 = sorted(Lk_minus_1)
    # Lk_minus_1 = [sorted(row) for row in Lk_minus_1]

    print(f"Lk_minus_1: {Lk_minus_1}")

    for i in range(len(Lk_minus_1)):
        for j in range(i + 1, len(Lk_minus_1)):
            l1 = list(Lk_minus_1[i])
            l2 = list(Lk_minus_1[j])

            print(f"l1: {l1}, l2: {l2}")

            if l1[:k_1-1] == l2[:k_1-1] and l1[k_1-1] < l2[k_1-1]:
                candidate = sorted(l1[:k_1-1] + [l1[k_1-1]] + [l2[k_1-1]])

                print(f"Candidate: {candidate}")

                if not has_infrequent_subset(candidate, Lk_minus_1):
                    candidates.append(candidate)
    return candidates

def count_support(transactions, itemset):
    count = 0
    itemset = set(itemset)
    for transaction in transactions:
        transaction_set = set(transaction)
        if itemset.issubset(transaction_set):
            count += 1
    return count

In [298]:
filepath = 'Datasets/sample.txt'
Transactions = read_dataset(filepath)
min_sup = 2

L = []

L1 = find_frequent_1_itemsets(Transactions, min_sup)

print(L1)

# print("Frequent 1-itemsets:")
# for itemset in L1:
#     print(itemset)
# The output will show the frequent 1-itemsets found in the dataset

L.append(L1)
Lk_minus_1 = L1
k = 2

# Ck = apriori_gen(Lk_minus_1, k - 1)
# print(f"Candidates for {k}-itemsets: {Ck}")
while Lk_minus_1:
    Ck = apriori_gen(Lk_minus_1, k - 1)
    print(f"Candidates for {k}-itemsets: {Ck}")
    Lk = []
    for candidate in Ck:
        if count_support(Transactions.values.tolist(), candidate) >= min_sup:
            Lk.append(frozenset(candidate))
    if not Lk:
        break
    L.append(Lk)
    Lk_minus_1 = set(Lk)
    
    print(f"#####  Frequent {k}-itemsets: {Lk}")

    k += 1


print(L)


[('I1',), ('I2',), ('I3',), ('I4',), ('I5',)]
Lk_minus_1: [('I1',), ('I2',), ('I3',), ('I4',), ('I5',)]
l1: ['I1'], l2: ['I2']
Candidate: ['I1', 'I2']
l1: ['I1'], l2: ['I3']
Candidate: ['I1', 'I3']
l1: ['I1'], l2: ['I4']
Candidate: ['I1', 'I4']
l1: ['I1'], l2: ['I5']
Candidate: ['I1', 'I5']
l1: ['I2'], l2: ['I3']
Candidate: ['I2', 'I3']
l1: ['I2'], l2: ['I4']
Candidate: ['I2', 'I4']
l1: ['I2'], l2: ['I5']
Candidate: ['I2', 'I5']
l1: ['I3'], l2: ['I4']
Candidate: ['I3', 'I4']
l1: ['I3'], l2: ['I5']
Candidate: ['I3', 'I5']
l1: ['I4'], l2: ['I5']
Candidate: ['I4', 'I5']
Candidates for 2-itemsets: []
[[('I1',), ('I2',), ('I3',), ('I4',), ('I5',)]]
